## RAG와 LLM을 활용한 질의응답 실습
- LLM API 호출 시 RAG를 통해 연관 정보 추출 후 Prompt에 추가하는 실습
- RAG 추가 전후로 평가 진행

In [1]:
import pickle

with open('./res/rag_data.pkl', 'rb') as f:
    rag_data = pickle.load(f)

In [2]:
questions = rag_data['questions'][:10]
contexts = rag_data['contexts'][:10]
answers = rag_data['answers'][:10]
len(questions)

10

## 평가
- RAGAS의 Answer Correctedness 평가 지표
  - 모델의 답변과 실제 정답 간의 일치율 측정


## Baseline (w/o RAG)


In [3]:
from tqdm import tqdm

from utils import call_openai, get_embeddings, cosine_similarity

predictions = []
for i in tqdm(range(len(questions))):
    prompt = f"""You are an expert in Finance. Please answer to the question given below.
    
Question:
{questions[i]}
"""

    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')
    predictions.append(prediction)

100%|██████████| 10/10 [01:05<00:00,  6.55s/it]


In [4]:
from datasets import Dataset 
from ragas import evaluate
from ragas.metrics import answer_correctness


data_samples = {
    'question': questions,
    'answer': predictions,
    'ground_truth': answers
}

dataset = Dataset.from_dict(data_samples)

score = evaluate(dataset, metrics=[answer_correctness])
score

/Users/hyeonjinho/Desktop/dev/Learned/llm/fastcampus-prompt-engineering-aillm-service/retrieval-augmented-generation/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Evaluating: 100%|██████████| 10/10 [00:33<00:00,  3.38s/it]


{'answer_correctness': 0.3855}

In [5]:
print(questions[0])

글로벌 저금리 현상이 부각된 원인은 무엇인가요?


In [6]:
print(predictions[0])

글로벌 저금리 현상이 부각된 원인에는 여러 가지 요인이 복합적으로 작용하고 있습니다. 주요 원인들을 살펴보면 다음과 같습니다:

1. **경제 성장 둔화**: 많은 선진국들이 경제 성장률이 둔화되고 있습니다. 경제 성장이 둔화되면 중앙은행들은 경제를 자극하기 위해 금리를 낮추는 경향이 있습니다.

2. **인구 고령화**: 인구 고령화는 저축률을 높이고 소비를 줄이는 경향이 있습니다. 이는 자본의 공급을 증가시키고, 자본에 대한 수요를 감소시켜 금리를 낮추는 요인으로 작용합니다.

3. **기술 발전**: 기술 발전은 생산성을 높이고, 비용을 절감하며, 인플레이션을 억제하는 효과가 있습니다. 이는 중앙은행들이 금리를 낮게 유지할 수 있는 환경을 조성합니다.

4. **글로벌화**: 글로벌화는 자본의 이동을 자유롭게 하고, 전 세계적으로 자본 공급을 증가시킵니다. 이는 금리를 낮추는 요인으로 작용합니다.

5. **중앙은행의 통화 정책**: 2008년 금융위기 이후 많은 중앙은행들이 양적 완화(QE)와 같은 비전통적인 통화 정책을 도입하여 금리를 낮추고 유동성을 공급했습니다. 이러한 정책들은 장기적으로 저금리 환경을 조성하는 데 기여했습니다.

6. **안전 자산에 대한 수요 증가**: 경제 불확실성과 금융 시장의 변동성 증가로 인해 투자자들은 안전 자산(예: 국채)에 대한 수요가 증가했습니다. 이는 국채 금리를 낮추는 요인으로 작용합니다.

7. **부채 수준 증가**: 많은 국가들이 높은 부채 수준을 유지하고 있으며, 이는 금리를 낮게 유지해야 하는 압력으로 작용합니다. 높은 금리는 부채 상환 부담을 증가시키기 때문입니다.

이러한 요인들이 복합적으로 작용하여 글로벌 저금리 현상이 지속되고 있습니다.


## LLM + RAG
- text-embedding-3-large 임베딩 모델 사용

In [7]:
import numpy as np
from tqdm import tqdm

from utils import call_openai, get_embeddings, cosine_similarity


def retrieve_context(question, contexts):
    question_embedding = get_embeddings([question], model='text-embedding-3-large')[0]
    context_embeddings = get_embeddings(contexts, model='text-embedding-3-large')

    similarities = [cosine_similarity(question_embedding, context_embedding) for context_embedding in context_embeddings]

    most_relevant_index = np.argmax(similarities)
    return contexts[most_relevant_index]

predictions = []
for i in tqdm(range(len(questions))):
    context = retrieve_context(questions[i], contexts[i])
    prompt = f"""You are an expert in Finance. Please answer to the question given below. Use information given in Context appropriately.

Context:
{context}

Question:
{questions[i]}
"""
    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')
    predictions.append(prediction)

100%|██████████| 10/10 [00:59<00:00,  5.96s/it]


In [8]:
from datasets import Dataset 
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness, context_recall, context_precision


data_samples = {
    'question': questions,
    'answer': predictions,
    'ground_truth': answers
}

dataset = Dataset.from_dict(data_samples)

score = evaluate(dataset, metrics=[answer_correctness])
score

ImportError: cannot import name 'context_relevancy' from 'ragas.metrics' (/Users/hyeonjinho/Desktop/dev/Learned/llm/fastcampus-prompt-engineering-aillm-service/retrieval-augmented-generation/.venv/lib/python3.11/site-packages/ragas/metrics/__init__.py)

In [ ]:
print(predictions[0])


In [ ]:
print(answers[0])